Task 1 - Install Spark, load required libraries, set environment variables, initiate Spark, load file

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
import findspark
findspark.init()



In [8]:
from pyspark import SparkContext

In [9]:
sc = SparkContext()

In [21]:
%%writefile example.txt
first 
second line 
the third line 
then a fourth line
and here is a fifth line 

Overwriting example.txt


In [22]:
textFile = sc.textFile('example.txt')

In [23]:
textFile.count()


5

In [24]:
textFile.first()

'first '

In [25]:
secfind = textFile.filter(lambda line: 'second' in line)

In [26]:
secfind

PythonRDD[10] at RDD at PythonRDD.scala:53

In [27]:
text_rdd = sc.textFile('example.txt')

In [28]:
words = text_rdd.map(lambda line: line.split())

In [29]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line'],
 ['and', 'here', 'is', 'a', 'fifth', 'line']]

In [30]:
text_rdd.collect()

['first ',
 'second line ',
 'the third line ',
 'then a fourth line',
 'and here is a fifth line ']

In [31]:
text_rdd.flatMap(lambda line : line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line',
 'and',
 'here',
 'is',
 'a',
 'fifth',
 'line']

In [33]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Overwriting services.txt


In [34]:
services = sc.textFile('services.txt')

In [35]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [37]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [41]:
clean = services.map(lambda line: line[1:] if line[0] == "#" else line)

In [42]:
clean = clean.map(lambda line: line.split())

In [43]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [45]:
pairs = clean.map(lambda lst: (lst[3], lst[-1]))

In [51]:
rekey = pairs.reduceByKey(lambda x,y: float(x) + float(y))

In [52]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [53]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [57]:
# State, Amount pairs
step1 = clean.map(lambda lst : (lst[3], lst[-1]))
# reduce by Key
step2 = step1.reduceByKey(lambda x, y : float(x) + float(y))
# get rid of State and Amount titles
step3 = step2.filter(lambda x: not x[0] == "State")
# sort the result
step4 = step3.sortBy(lambda x: x[1], ascending=False)
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [58]:
x = ["ID", "State", "Amount"]
